In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth [colab-new] @ git+https://github.com/unslothai/unsloth.git"
# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
!pip install transformers

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Ania ang usa ka instruksyon nga naghulagway sa usa ka buluhaton, gipares sa usa ka input nga naghatag og dugang nga konteksto. Paghimo og tubag nga angayng makompleto sa hangyo.

### Instruksiyon:
{}

### Input:
{}

### Tubag:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    print(examples)
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset, concatenate_datasets
dataset_cebuano_data = load_dataset("json", data_files=f'datasets/chatgpt_cebuano_data.jsonl', split='train')
dataset_cebuano_translation = load_dataset("json", data_files=f'datasets/chatgpt_cebuano_translation.jsonl', split='train')
dataset_cebuano_story = load_dataset("json", data_files=f'datasets/chatgpt_cebuano_story.jsonl', split='train')
combined_dataset = concatenate_datasets([dataset_cebuano_data, dataset_cebuano_translation, dataset_cebuano_story]).shuffle(seed=32)
dataset = combined_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/5225 [00:00<?, ? examples/s]

{'instruction': ['Pagsulat ug usa ka istorya kabahin sa Pag-respeto sa mga ginikanan', 'Unsa ang gibuhat sa mga mag-uuma sa umahan?', 'Kanus-a magsugod si Jake sa iyang klase?', 'Pagsulat ug usa ka istorya', 'Pagsulat ug usa ka istorya kabahin sa Paghatag og Bulak sa Usa ka Higala', "Unsa ang Cebuano sa 'Roof'", 'Pagsulat ug usa ka istorya pang grade 1.', 'Continue the following sentence in Cebuano.', 'Pagsulat ug usa ka istorya', 'Kanus-a magdula si Niko ug ang iyang mga kauban?', 'Pagsulat ug usa ka istorya', 'Asa magsugod si Lily magbuhat og experiment?', 'Pagsulat ug usa ka istorya kabahin sa Pagkaon sa Himsog nga Pagkaon', 'Translate to Cebuano: I felt excited when I saw the birthday cake.', 'Giunsa ni Jojo ang pagtabang sa iyang kuya?', 'Pagsulat ug usa ka istorya', 'Pagsulat ug usa ka istorya kabahin sa Pagpanarbaho', 'Continue the following sentence in Cebuano.', 'Pagsulat ug usa ka istorya', 'Pagsulat ug usa ka istorya', 'Continue the following sentence in Cebuano.', "Unsa ang

In [ ]:
dataset[0]

{'instruction': 'Pagsulat ug usa ka istorya kabahin sa Pag-respeto sa mga ginikanan',
 'input': '',
 'output': 'Titulo: Ang Respeto ni Liza sa mga Ginikanan\n\nIstorya: Si Liza nagpasalamat sa iyang mga ginikanan tungod sa tanan nilang sacrifices. Siya magpasalamat bisan sa mga gamay nga butang ug magpakita og respeto sa ilang mga sugo. Si Liza nakat-on nga ang respeto sa mga ginikanan usa ka dakong bahin sa pagkahimong maayo nga tawo.\n\n1. Kinsa ang nagpasalamat sa mga ginikanan?\nKapilian: A. Liza B. Toto C. Ana D. Lito\nTubag: A\n2. Unsa ang gibuhat ni Liza?\nKapilian: A. Nagpasalamat B. Nagbasa C. Nagsayaw D. Nagtan-aw og TV\nTubag: A\n3. Asa siya nagpasalamat?\nKapilian: A. Sa balay B. Sa parke C. Sa simbahan D. Sa eskwelahan\nTubag: A\n4. Unsa ang nakat-onan ni Liza?\nKapilian: A. Magpasagad B. Magpasalamat ug magrespeto sa ginikanan C. Magdula D. Magtan-aw og TV\nTubag: B\n5. Unsa ang epekto sa respeto sa mga ginikanan?\nKapilian: A. Pagkahimong maayo nga tawo B. Kasuko C. Kapo

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,225 | Num Epochs = 1 | Total steps = 653
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,0.393400
2,0.382000
3,0.333700
4,0.449900
5,0.340300
6,0.402700
7,0.317900
8,0.364300
9,0.443500
10,0.547400


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "cebuano_lora_epoch_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
if True:
    alpaca_prompt = """Ania ang usa ka instruksyon nga naghulagway sa usa ka buluhaton, gipares sa usa ka input nga naghatag og dugang nga konteksto. Paghimo og tubag nga angayng makompleto sa hangyo.

### Instruksiyon:
{}

### Input:
{}

### Tubag:
{}
"""

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Pagsulat ug usa ka istorya bahin ni Juan og iyang mga amigo.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2000, do_sample=True, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ania ang usa ka instruksyon nga naghulagway sa usa ka buluhaton, gipares sa usa ka input nga naghatag og dugang nga konteksto. Paghimo og tubag nga angayng makompleto sa hangyo.\n\n### Instruksiyon:\nPagsulat ug usa ka istorya bahin ni Juan og iyang mga amigo.\n\n### Input:\n\n\n### Tubag:\n\nTitulo: Ang Pagdula ni Juan ug ang iyang mga Amigo\nIstroya: Si Juan ug ang iyang mga amigo nagdula sa parke. Nagdula sila og tago-taguan ug nagpasalamat sa kalipay nga nadawat sa dula. Ang moral sa istorya mao nga ang pagdula sa mga amigo maghatag og kalipay ug magpahimulos sa tanan.\n1. Unsa ang gibuhat ni Juan ug ang iyang mga amigo? \nKapilian: A. Nagdula og tago-taguan \nB. Nagtrabaho \nC. Nagluto \nD. Nagbasa og libro \nTubag: A\n2. Ngano nga nagdula si Juan ug ang iyang mga amigo? \nKapilian: A. Kay gusto sila magpasalamat sa kalipay \nB. Kay gusto sila magtrabaho \nC. Kay gusto sila magluto \nD. Kay gusto sila magbasa og libro \nTubag: A\n3. Asa nagdula si Juan ug ang iy

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Pagsulat ug usa ka istorya bahin ni Ian og iyang iring.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2000)

<|begin_of_text|>Ania ang usa ka instruksyon nga naghulagway sa usa ka buluhaton, gipares sa usa ka input nga naghatag og dugang nga konteksto. Paghimo og tubag nga angayng makompleto sa hangyo.

### Instruksiyon:
Pagsulat ug usa ka istorya bahin ni Ian og iyang iring.

### Input:


### Tubag:

Titulo: Ang Iro ni Ian
Istroya: Si Ian adunay iro nga si Max. Kada buntag, siya magdula uban ni Max sa ilang bakuran. Ang ilang kalipay magpabilin hangtod sa hapon. Ang moral sa istorya mao nga ang mga hayop maghatag og kalipay sa pamilya.
1. Unsa ang pangalan sa iro ni Ian? 
Kapilian: A. Max 
B. Mia 
C. Toto 
D. Benji 
Tubag: A
2. Asa magdula si Ian ug Max? 
Kapilian: A. Sa bakuran 
B. Sa parke 
C. Sa simbahan 
D. Sa eskwelahan 
Tubag: A
3. Unsa ang gibati ni Ian sa iyang iro? 
Kapilian: A. Kalipay 
B. Kasuko 
C. Pagkapoy 
D. Pagkalisod 
Tubag: A
4. Unsa ang tema sa istorya? 
Kapilian: A. Pag-atiman sa iro 
B. Pag-away sa mga bata 
C. Pagpanikas sa pamilya 
D. Pagpanikas sa kalikupan 
Tubag: A


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Translate to English: 'Si Ian adunay iring nga si Bibo. Kada buntag, magdula sila og bola sa gawas. Ang iring magdali ug magdula, ug si Ian magpasalamat sa iyang kalipay. Ang moral sa istorya mao nga ang pag-atiman sa mga hayop maghatag og kalipay.'", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2000)

<|begin_of_text|>Ania ang usa ka instruksyon nga naghulagway sa usa ka buluhaton, gipares sa usa ka input nga naghatag og dugang nga konteksto. Paghimo og tubag nga angayng makompleto sa hangyo.

### Instruksiyon:
Translate to English: 'Si Ian adunay iring nga si Bibo. Kada buntag, magdula sila og bola sa gawas. Ang iring magdali ug magdula, ug si Ian magpasalamat sa iyang kalipay. Ang moral sa istorya mao nga ang pag-atiman sa mga hayop maghatag og kalipay.'

### Input:


### Tubag:

Ian had a dog named Bibo. Every morning, they would play with a ball outside. The dog would run and play, and Ian would be thankful for the joy. The moral of the story is that taking care of animals brings joy.
<|end_of_text|>


In [ ]:
model.save_pretrained("cebuano_lora_epoch_model")
tokenizer.save_pretrained("cebuano_lora_epoch_model")

('cebuano_lora_epoch_model/tokenizer_config.json',
 'cebuano_lora_epoch_model/special_tokens_map.json',
 'cebuano_lora_epoch_model/tokenizer.json')

In [ ]:
model.save_pretrained_gguf("model_epoch", tokenizer,)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.26 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 44%|████▍     | 14/32 [00:01<00:01, 12.39it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:25<00:00,  2.66s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model_epoch/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model_epoch/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model_epoch/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model_epoch/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model_epoch into q8_0 GGUF format.
The output location will be /content/model_epoch/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model_epoch
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-